<a href="https://colab.research.google.com/github/pertvirt/hello_world/blob/master/TF_IDF_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Document Retrieval using TF-IDF Weighted Rank and TF-IDF Cosine Similarity

## Imports

In [0]:
!unzip stories

In [2]:
!pip install num2words

     |████████████████████████████████| 102kB 2.8MB/s 


In [0]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

# %load_ext autotime

In [0]:
title = "data"
alpha = 0.3

## Taking all folders

In [0]:
folders = [x[0] for x in os.walk(str(os.getcwd())+'/'+title+'/')]
folders[0] = folders[0][:len(folders[0])-1]

In [31]:
folders

['/content/data', '/content/data/.ipynb_checkpoints']

In [0]:
for i in folders:
  print(str(i))

## Collecting the file names and titles

In [46]:
import glob
l = glob.glob("data/*.txt")
dataset = []
for i in l:
  # dataset.append(i.replace("data/", ""))
  dataset.append(i) 

print(dataset)
print(len(dataset))

['data/Control your smart home with the Google Assistant.txt', 'data/Can Facebook And Google Detect And Stop Deepfakes?.txt', 'data/Prepare for Your Google Interview: Leadership.txt', 'data/Stonewall Forever - A project by The Center with support from Google.txt', 'data/Google at 2019 Milan Design Week: A Space for Being.txt', 'data/Introducing Google Input Tools.txt', 'data/Live Caption: Making digital media more accessible with a single tap.txt', 'data/Project Euphonia: Helping everyone be better understood.txt', 'data/DIVA: Making the Google Assistant more accessible.txt', 'data/Take Your Child to Work Day at Google 2019.txt', 'data/The Try Guys Try 13 Future Technologies At Google.txt', 'data/Google Apps Quick Tour.txt', 'data/Create Your Resume for Google: Tips and Advice.txt', 'data/How Search Works.txt', 'data/Real Talk with Google Software Engineer.txt', 'data/What is Google’s Advanced Protection Program?.txt', 'data/Using Android’s Digital Wellbeing tools to spend less time on

In [24]:
# dataset = []

# c = False

# for i in folders:
#     file = open(i, 'r')
#     text = file.read().strip()
#     file.close()

#     file_name = re.findall('><A HREF="(.*)">', text)
#     file_title = re.findall('<BR><TD> (.*)\n', text)

#     if c == False:
#         file_name = file_name[2:]
#         c = True
        
#     print(len(file_name), len(file_title))

#     for j in range(len(file_name)):
#         dataset.append((str(i) +"/"+ str(file_name[j]), file_title[j]))

#     print(file_name)

452 452
['100west.txt', '13chil.txt', '14.lws', '16.lws', '17.lws', '18.lws', '19.lws', '20.lws', '3gables.txt', '3lpigs.txt', '3sonnets.vrs', '3student.txt', '3wishes.txt', '4moons.txt', '5orange.txt', '6ablemen.txt', '6napolen.txt', '7oldsamr.txt', '7voysinb.txt', 'ab40thv.txt', 'abbey.txt', 'abyss.txt', 'adler.txt', 'adv_alad.txt', 'advsayed.txt', 'advtthum.txt', 'aesop11.txt', 'aesopa10.txt', 'aircon.txt', 'aisle.six', 'aislesix.txt', 'alad10.txt', 'alissadl.txt', 'altside.hum', 'aluminum.hum', 'aminegg.txt', 'angelfur.hum', 'angry_ca.txt', 'antcrick.txt', 'aquith.txt', 'arcadia.sty', 'archive', 'arctic.txt', 'asop', 'assorted.txt', 'bagel.man', 'bagelman.txt', 'batlslau.txt', 'beast.asc', 'beautbst.txt', 'beggars.txt', 'bern', 'berternie.txt', 'bestwish', 'beyond.hum', 'bgb.txt', 'bgcspoof.txt', 'bigred.hum', 'bishop00.txt', 'blabnove.hum', 'blabnove.txt', 'blackp.txt', 'blackrdr', 'blak', 'blasters.fic', 'blh.txt', 'blind.txt', 'blossom.pom', 'blue', 'bluebrd.txt', 'bookem.1', 'b

In [47]:
len(dataset)

28

In [0]:
N = len (dataset)

In [0]:
def print_doc(id):
    print(dataset[id])
    file = open(dataset[id], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)

# Preprocessing

In [0]:
def convert_lower_case(data):
    return np.char.lower(data)

In [0]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [0]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [0]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [0]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [0]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [0]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

## Extracting Data

In [59]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
processed_text = []
# processed_title = []

for i in dataset[:N]:
    # file = open(i[0], 'r', encoding="utf8", errors='ignore')
    file = open(i, 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))
    # processed_title.append(word_tokenize(str(preprocess(i[1]))))

## Calculating DF for all words

In [0]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    # tokens = processed_title[i]
    # for w in tokens:
    #     try:
    #         DF[w].add(i)
    #     except:
    #         DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [0]:
DF

In [0]:
total_vocab_size = len(DF)

In [64]:
total_vocab_size

2149

In [0]:
total_vocab = [x for x in DF]

In [66]:
print(total_vocab[:20])

['zero', 'five', 'thousand', 'seven', 'hundr', 'sixti', 'eight', 'forti', 'speaker', 'googl', 'assist', 'access', 'video', 'ten', 'two', 'control', 'smart', 'home', 'twelv', 'requir']


In [0]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

### Calculating TF-IDF for body, we will consider this as the actual tf-idf as we will add the title weight to this.

In [0]:
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens)
    words_count = len(tokens)
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

In [69]:
tf_idf

{(0, 'access'): 0.00372276481465884,
 (0, 'account'): 0.0006432838602930261,
 (0, 'action'): 0.0008780509078683185,
 (0, 'also'): 0.0002667744684966323,
 (0, 'anoth'): 0.00040077246379866997,
 (0, 'app'): 0.0011689023502999547,
 (0, 'approach'): 0.0008780509078683185,
 (0, 'around'): 0.00044075383626297385,
 (0, 'ask'): 0.0007275109893818336,
 (0, 'assign'): 0.0013357385861271372,
 (0, 'assist'): 0.021951272696707963,
 (0, 'awar'): 0.001133208562097085,
 (0, 'away'): 0.0015739623983600588,
 (0, 'blind'): 0.001133208562097085,
 (0, 'case'): 0.0005318235449512629,
 (0, 'categori'): 0.0009895112232100815,
 (0, 'check'): 0.0006432838602930261,
 (0, 'chromecast'): 0.001133208562097085,
 (0, 'clari'): 0.013357385861271373,
 (0, 'code'): 0.0005844511751499774,
 (0, 'coffeemak'): 0.0013357385861271372,
 (0, 'colleagu'): 0.001133208562097085,
 (0, 'com'): 0.001133208562097085,
 (0, 'command'): 0.0013357385861271372,
 (0, 'comment'): 0.001133208562097085,
 (0, 'compat'): 0.0013357385861271372,
 

### Calculating TF-IDF for Title

In [0]:
# doc = 0

# tf_idf_title = {}

# for i in range(N):
    
#     tokens = processed_title[i]
#     counter = Counter(tokens + processed_text[i])
#     words_count = len(tokens + processed_text[i])

#     for token in np.unique(tokens):
        
#         tf = counter[token]/words_count
#         df = doc_freq(token)
#         idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
#         tf_idf_title[doc, token] = tf*idf

#     doc += 1

In [0]:
# tf_idf_title

In [71]:
tf_idf[(0,"us")]

0.0006585870418424215

In [0]:
# tf_idf_title[(0,"go")]

0.0002906893990853149

## Merging the TF-IDF according to weights

In [0]:
for i in tf_idf:
    tf_idf[i] *= alpha

In [0]:
# for i in tf_idf_title:
#     tf_idf[i] = tf_idf_title[i]

In [73]:
len(tf_idf)

6344

# TF-IDF Matching Score Ranking

In [78]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    l = []
    
    for i in query_weights[:10]:
        l.append(i[0])
    
    print(l)
    

matching_score(5, "millions of Americans with hearing")

Matching Score

Query: millions of Americans with hearing

['million', 'american', 'hear']

[6, 21, 26, 3, 1, 23, 2, 20, 0, 24]


In [0]:
print_doc(21)

# TF-IDF Cosine Similarity Ranking

In [0]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

### Vectorising tf-idf

In [0]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [0]:
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [85]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    print(out)

#     for i in out:
#         print(i, dataset[i][0])

Q = cosine_similarity(5, "inventors of UNIX")

Cosine Similarity

Query: inventors of UNIX

['inventor', 'unix']

[20 27 12  1  2]


In [86]:
print_doc(20)

data/Working at Google London.txt
1
00:00:01,940 --> 00:00:08,589
[Music]

2
00:00:09,240 --> 00:00:13,720
so London is one of our core development

3
00:00:11,920 --> 00:00:16,240
centers for mobile technologies the

4
00:00:13,720 --> 00:00:18,400
London office is a fairly young Google

5
00:00:16,240 --> 00:00:20,890
office it's a full fully fledged Google

6
00:00:18,400 --> 00:00:22,720
office so it has engineering and sales

7
00:00:20,890 --> 00:00:24,910
stuff so what we try to do is we try to

8
00:00:22,720 --> 00:00:26,770
build a really strong mobile engineering

9
00:00:24,910 --> 00:00:28,390
effort based in London that can really

10
00:00:26,770 --> 00:00:34,059
drive on mobile initiatives around the

11
00:00:28,390 --> 00:00:35,769
world working at Google is unique in a

12
00:00:34,059 --> 00:00:37,329
lot of ways the first day I was there

13
00:00:35,769 --> 00:00:39,489
had an email asking if I wanted to break

14
00:00:37,329 --> 00:00:41,319
in for a free messag

('/Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/ghost', 'Time for Flowers, by Gay Bost')
TIME FOR FLOWERS
  by Gay Bost

They'd put flowers up. She hadn't noticed. Time wouldn't hold still.
She remembered, quite clearly, that time had been a simple thing; one
moment following the previous one, seconds strung out neatly like her
mother's pearls laid out on the dark mahogany vanity each Sunday
morning. But there had been a catch . . . 

Hung around Mother's neck the catch clicked and the tidy little line 
of seconds became a never ending circle with only the catch in the 
middle. For some reason the thought of pearls gathered from the sea, 
naturally nested within the confines of oyster shells, scattered 
haphazardly about the ocean floor disturbed her.

Now they'd put up the flowers in the same careless groupings. This,
too, disturbed her. Bright yellow trumpets, their collars spread to
catch the sun, dotted the front yard in clusters of two or three, five
or six. Bunc